# 10.3 透過生命週期驗證參考

Ref: [透過生命週期驗證參考](https://rust-lang.tw/book-tw/ch10-03-lifetime-syntax.html)

> 生命週期（lifetime）是另一種我們已經使用過的泛型。不同於確保一個型別有沒有我們要的行為，生命週期確保我們在需要參考的時候，它們都是有效的。
>
> 我們在第四章的「參考與借用」段落沒談到的是，Rust 中的每個參考都有個生命週期，這是決定該參考是否有效的作用域。大多情況下生命週期是隱式且可推導出來的，就像大多情況下型別是可推導出來的。當多種型別都有可能時，我們就得詮釋型別。同樣地，當生命週期的參考能以不同方式關聯的話，我們就得詮釋生命週期。Rust 要求我們用泛型生命週期參數來詮釋參考之間的關係，以確保實際在執行時的參考絕對是有效的。
>
> 詮釋生命週期在大多數的程式語言中都沒有這個概念，所以這段可能會有點讓你覺得陌生。雖然我們不會在此章涵蓋所有生命週期的內容，但是我們會講些你可能遇到生命週期的常見場景，好讓你更加熟悉這個概念。

## 10.3.1 透過生命週期預防迷途參考

> 生命週期最主要的目的就是要預防迷途參考（dangling references），其會導致程式參考到其他資料，而非它原本想要的參考。請看一下下面的程式，它有一個外部作用域與內部作用域。
>

<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

In [31]:
// 範例 10-16：嘗試使用其值已經離開作用域的參考
fn main() {
    let r;

    {
        let x = 5;
        r = &x;
    }

    println!("r: {}", r);
}

Error: `x` does not live long enough

## 10.3.2 借用檢查器

> Rust 編譯器有個借用檢查器（borrow checker）會比較作用域來檢測所有的借用是否有效。下方顯示了上方的程式碼，但加上了變數生命週期的詮釋。

如果有用過Python的人，應該知道local variable和global variable的差別，這邊的生命週期就是類似的概念


<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

In [56]:
// 範例 10-17：變數 r 與 x 的生命週期詮釋，分別以 'a 和 'b 作為表示
fn main() {
    let r;                // ---------+-- 'a
                          //          |
    {                     //          |
        let x = 5;        // -+-- 'b  |
        r = &x;           //  |       |
    }                     // -+       |
                          //          |
    println!("r: {}", r); //          |
}                         // ---------+


Error: `x` does not live long enough

這邊原先的 `x` 以我個人從Python的角度來看就是像是一個階層更裡面的local variables

```py
def main():
    r = None
    def inner():
        x: int = 5
        r = 5
        print(r)
    inner()
    print(r)
```

在Python中會這樣不對`r`起真正作用，而Rust因這是個結束生命週期的存在直接報錯
```py
>>> main()
5
None
```

所以我們得把 `x` 帶到外面來

In [ ]:
// 範例 10-18：一個有效參考，因為資料比參考的生命週期還長
fn main() {
    let x = 5;            // ----------+-- 'b
                          //           |
    let r = &x;           // --+-- 'a  |
                          //   |       |
    println!("r: {}", r); //   |       |
                          // --+       |
}                         // ----------+


## 10.3.3 函式中的泛型生命週期

> 讓我們寫個回傳兩個字串切片中較長者的函式。此函式會接收兩個字串切片並回傳一個字串切片。在我們實作 longest 函式後，下列的程式碼應該要印出 最長的字串為 abcd。

In [32]:
// 範例 10-19：main 函式呼叫 longest 函式來找出兩個字串切片中較長的
fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";

    let result = longest(string1.as_str(), string2);
    println!("最長的字串為 {}", result);
}


Error: cannot find function `longest` in this scope

<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

In [34]:
fn longest(x: &str, y: &str) -> &str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

Error: missing lifetime specifier

> 提示文字表示回傳型別需要有一個泛型生命週期參數，因為 Rust 無法辨別出回傳的參考指的是 x 還是 y。事實上，我們也不知道，因為函式本體中的 if 區塊會回傳 x 的參考而 else 區塊會回傳 y 的參考！
>
> 當我們定義函式時，我們不知道傳遞進此函式的實際數值會是什麼，所以我們不知道到底是 if 或 else 的區塊會被執行。我們也不知道傳遞進來的參考實際的生命週期為何，所以我們無法像範例 10-17 和 10-18 
>
> 10-17: 
> 
> ```rs
> fn main() {
>     let x = 5;            // ----------+-- 'b
>                           //           |
>     let r = &x;           // --+-- 'a  |
>                           //   |       |
>     println!("r: {}", r); //   |       |
>                           // --+       |
> }    
> ```
> 那樣觀察作用域，來判定我們回傳的參考會永遠有效。要修正此錯誤，我們要加上泛型生命週期參數來定義參考之間的關係，讓借用檢查器能夠進行分析。



## 10.3.4 生命週期詮釋語法

> 生命週期詮釋（Lifetime Annotation）不會改變參考能存活多久，它們僅描述了數個參考的生命週期之間互相的關係，而不會影響其生命週期。就像當函式簽名指定了一個泛型型別參數時，函式便能夠接受任意型別一樣。函式可以指定一個泛型生命週期參數，這樣函式就能接受任何生命週期。
>
> 生命週期詮釋的語法有一點不一樣：生命週期參數的名稱必須以撇號（`'`）作為開頭，通常全是小寫且很短，就像泛型型別一樣。大多數的人會使用名稱 `'a` 作為第一個生命週期詮釋。我們將生命週期參數置於參考的 `&` 之後，並使用空格區隔詮釋與參考的型別。
>
> 以下是一些例子：沒有生命週期參數的 `i32` 參考、有生命週期 `'a` 的 `i32` 參考以及有生命週期 `'a` 的 `i32` 可變參考。
>


```rs
&i32        // 一個參考
&'a i32     // 一個有顯式生命週期的參考
&'a mut i32 // 一個有顯式生命週期的可變參考
```

#### 10.3.5 函式簽名中的生命週期詮釋

> 要在函式簽名使用生命週期詮釋的話，我們需要在函式名稱與參數列表之間的尖括號內宣告泛型生命週期參數，就像泛型型別參數那樣。
>
> 我們想在此簽名表達這樣的限制：只要所有參數都要是有效的，那麼回傳的參考才也會是有效的。也就是參數的生命週期與回傳參考的生命週期是相關的。我們會將生命週期命名為 'a 然後將它加到每個參考，如範例 10-21 所示。

- 可以相對簡易的理解為，你事先透過註解生命週期，編譯器起碼知道裡面的參數是什麼生命週期，而不是完全不知道，而遭遇生命週期問題帶來的麻煩

In [35]:
// 範例 10-21：longest 函式定義指定所有簽名中的參考必須有相同的生命週期 'a
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

In [36]:
fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";

    let result = longest(string1.as_str(), string2);
    println!("最長的字串為 {}", result);
}
main()

最長的字串為 abcd


()

- 這邊我們給了 `longest` 兩個來自不同生命週期的變數，但是 `string1` 來自外層較長生命週期的位置，而執行 `longest` 的位置以及加入比較的 `string2` 來自內部短生命週期的地方，但因為 `string1` 在裡面這個位置都能全程參與運行，所以也視同一個內部短週期的一員。


In [38]:
// 範例 10-22 使用 longest 函式並傳入 String 數值的參考，但兩個參數的實際生命週期均不相同
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn main() {
    let string1 = String::from("很長的長字串");

    {
        let string2 = String::from("xyz");
        let result = longest(string1.as_str(), string2.as_str());
        println!("最長的字串為 {}", result);
    }
}
main()

最長的字串為 很長的長字串


()

- 但是在下面這個範例裡，我們是先將到接收結果的 `result` 先定義在了外面的長週期，但是 `longest` 運算的結果屬於短週期。當短週期結束時，本來`result`的結果就失效了，所以這樣就會出現問題。
<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

In [40]:
// 範例 10-23：嘗試在 string2 離開作用域後使用 result
fn main() {
    let string1 = String::from("很長的長字串");
    let result;
    {
        let string2 = String::from("xyz");
        result = longest(string1.as_str(), string2.as_str());
    }
    println!("最長的字串為 {}", result);
}

fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}


Error: `string2` does not live long enough

## 10.3.6 深入理解生命週期

以我自己的理解來看，Rust將生命週期的概念也當做了一種類似於型別的存在處理

所以設計了能透過相當於型別註解的方式，來讓編譯器知道這個變數的生命週期是什麼

> 你要指定生命週期參數的方式取決於函式的行為。舉例來說如果我們改變函式 longest 的實作為永遠只回傳第一個參數而不是最長的字串切片，我們就不需要在參數 y 指定生命週期。以下的程式碼就能編譯
>

- 由於這邊的 `longest` 只會回傳 `x`， `y` 的生命週期就不會影響到 `longest` 的回傳值，所以就不需要指定生命週期了。

In [41]:
fn main() {
    let string1 = String::from("abcd");
    let string2 = "efghijklmnopqrstuvwxyz";

    let result = longest(string1.as_str(), string2);
    println!("最長的字串為 {}", result);
}

fn longest<'a>(x: &'a str, y: &str) -> &'a str {
    x
}


- 下面這邊因為連回傳的值都與本來傳入的生命週期無關，如果這個時候還定義回傳值的生命週期，那麼就會出現錯誤。


In [42]:
fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";

    let result = longest(string1.as_str(), string2);
    println!("最長的字串為 {}", result);
}

fn longest<'a>(x: &str, y: &str) -> &'a str {
    let result = String::from("超長的字串");
    result.as_str()
}


Error: unused variable: `x`

Error: unused variable: `y`

Error: cannot return reference to local variable `result`

## 10.3.7 結構體定義中的生命週期詮釋

類似於我們註解範型型別的方式，我們也可以在結構體中註解生命週期

In [43]:
// 範例 10-24：擁有參考的結構體需要加上生命週期詮釋
struct ImportantExcerpt<'a> {
    part: &'a str,
}

fn main() {
    let novel = String::from("Call me Ishmael. Some years ago...");
    let first_sentence = novel.split('.').next().expect("無法找到 '.'");
    let i = ImportantExcerpt {
        part: first_sentence,
    };
}
main()

()

## 10.3.8 生命週期省略

這邊就有點提到Rust的歷史

> 你已經學到了每個參考都有個生命週期，而且你需要在有使用參考的函式與結構體中指定生命週期參數。然而在第四章的範例 4-9 我們有函式可以不詮釋生命週期並照樣編譯成功。

In [44]:
// 範例 10-25：在範例 4-9 定義過的函式，雖然其參數與回傳值均為參考，卻仍可編譯成功
fn first_word(s: &str) -> &str {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }

    &s[..]
}

fn main() {
    let my_string = String::from("hello world");

    // first_word 能用在`String` 的切片
    let word = first_word(&my_string[..]);

    let my_string_literal = "hello world";

    // first_word 能用在字串字面值
    let word = first_word(&my_string_literal[..]);

    // 因為字串字面值已經是字串切片了
    // 所以也可以不用加上字串語法！
    let word = first_word(my_string_literal);
}
main()

()

在早期的版本，你必須向寫好型別註解一樣，對每個參考都要註解生命週期，但是這樣會造成很多重複的註解，所以Rust就設計了一些規則，讓編譯器可以自動推斷生命週期，而不需要你手動註解。

不然你本來要這樣
```rs
fn first_word<'a>(s: &'a str) -> &'a str {
```

## 10.3.9 靜態生命週期

基本上類似於global variable的概念，在程式運行都會存在的生命週期

> 其中有個特殊的生命週期 'static 我們需要進一步討論，這是指該參考可以存活在整個程式期間。所有的字串字面值都有 'static 生命週期，我們可以這樣詮釋：

In [74]:
let s: &'static str = "我有靜態生命週期。";

---
## 總結 - 組合泛型型別參數、特徵界限與生命週期

> 讓我們用一個函式來總結泛型型別參數、特徵界限與生命週期的語法！

現在你學會了所有型別以及生命週期註解的方法，可以試試寫出來，來總結這些概念

In [ ]:
fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";

    let result = longest_with_an_announcement(
        string1.as_str(),
        string2,
        "Today is someone's birthday!",
    );
    println!("最長的字串為 {}", result);
}

use std::fmt::Display;

fn longest_with_an_announcement<'a, T>(
    x: &'a str,
    y: &'a str,
    ann: T,
) -> &'a str
where
    T: Display,
{
    println!("公告！{}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}
